In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
def extract_timestamp(filename):
    """
    Extract timestamp from the filename.
    Example of OMI file: omi_l2_v2+soft+cmc_2023m0502_v0401_BY2_S05N65.nc
    Example of IAGOS file: IAGOS_profile_2023050203041714_L2_3.1.0-descent.nc4
    """
    # For IAGOS files
    if 'IAGOS' in filename:
        # Match 16-digit timestamp
        match = re.search(r'IAGOS_profile_(\d{16})', filename)
        if match:
            try:
                timestamp_str = match.group(1)
                # Parse the first 14 digits as YYYYMMDDHHMMSS, ignore the last two digits
                timestamp = datetime.strptime(timestamp_str[:14], '%Y%m%d%H%M%S')
                return timestamp
            except ValueError:
                print(f"Unable to parse timestamp in IAGOS file: {match.group(1)}")
                return None
    # For OMI files
    elif 'omi' in filename:
        # Match date information
        match = re.search(r'(\d{4})m(\d{4})', filename)
        if match:
            year, month_day = match.groups()
            month = month_day[:2]
            day = month_day[2:]
            date_str = f"{year}{month}{day}"
            try:
                # Assume the OMI file time is 04:30 on that day
                timestamp = datetime.strptime(date_str, '%Y%m%d') + timedelta(hours=4, minutes=30)
                return timestamp
            except ValueError:
                print(f"Unable to parse date in OMI file: {date_str}")
                return None
    return None

def find_closest_iagos_file(omi_time, iagos_files):
    """
    Find the IAGOS file closest in time to the given OMI time.
    """
    min_diff = float('inf')
    best_match = None
    for f in iagos_files:
        iagos_time = extract_timestamp(f)
        if iagos_time:
            time_diff = abs((omi_time - iagos_time).total_seconds())
            if time_diff < min_diff:
                min_diff = time_diff
                best_match = f
    return best_match


In [ ]:
def process_iagos_data(iagos_file_path, pressure_boundaries_omi, omi_height_layers):
    """
    Process IAGOS data, calculating the average PPB and corresponding DU within each OMI pressure layer.
    """
    try:
        ds_iagos = xr.open_dataset(iagos_file_path)
    except Exception as e:
        print(f"      Error: Unable to open IAGOS file {iagos_file_path}: {e}")
        return None, None, None

    # Extract PPB and pressure data from IAGOS data
    try:
        ppb_data = ds_iagos['O3_P1'].values  # Ozone concentration (ppb)
        pressure_data = ds_iagos['air_press_AC'].values / 100  # Convert pressure from Pa to hPa
    except KeyError as e:
        print(f"      Error: Missing necessary variable {e} in IAGOS file {iagos_file_path}.")
        return None, None, None

    # Read 'O3_P1_validity_flag' and apply filtering
    try:
        ozone_flag = ds_iagos['O3_P1_validity_flag'].values  # Validity flag for ozone concentration
    except KeyError as e:
        print(f"      Error: Missing necessary variable {e} in IAGOS file {iagos_file_path}.")
        return None, None, None

    # Keep only points where validity flag is 0
    valid_idx = ozone_flag == 0  # Keep only valid points

    # Apply filter to ppb_data and pressure_data
    ppb_data = ppb_data[valid_idx]
    pressure_data = pressure_data[valid_idx]

    print(f"      Processing IAGOS file: {os.path.basename(iagos_file_path)}")
    print(f"        Original PPB data length: {len(ppb_data)}, Original pressure data length: {len(pressure_data)}")

    # Save original IAGOS data to CSV
    iagos_filename = os.path.basename(iagos_file_path)
    iagos_timestamp = extract_timestamp(iagos_filename)
    iagos_datetime_str = iagos_timestamp.strftime("%Y%m%d%H%M%S") if iagos_timestamp else 'unknown'

    df_iagos_original = pd.DataFrame({
        'Ozone_Concentration_ppb': ppb_data,
        'Pressure_hPa': pressure_data,
    })
    iagos_original_csv_filename = f'IAGOS_Original_Data_{iagos_datetime_str}.csv'
    iagos_original_csv_path = os.path.join(csv_output_dir, iagos_original_csv_filename)
    df_iagos_original.to_csv(iagos_original_csv_path, index=False)
    print(f"      Original IAGOS data saved to {iagos_original_csv_path}")

    # Remove NaN values
    mask_valid = ~np.isnan(ppb_data) & ~np.isnan(pressure_data)
    ppb_values = ppb_data[mask_valid]
    pressure_values = pressure_data[mask_valid]

    print(f"        After removing NaN, PPB data length: {len(ppb_values)}, Pressure data length: {len(pressure_values)}")
    print(f"        IAGOS pressure range: {pressure_values.min()} - {pressure_values.max()} hPa")

    # Save valid IAGOS data to CSV
    df_iagos_valid = pd.DataFrame({
        'Ozone_Concentration_ppb': ppb_values,
        'Pressure_hPa': pressure_values,
    })
    iagos_valid_csv_filename = f'IAGOS_Valid_Data_{iagos_datetime_str}.csv'
    iagos_valid_csv_path = os.path.join(csv_output_dir, iagos_valid_csv_filename)
    df_iagos_valid.to_csv(iagos_valid_csv_path, index=False)
    print(f"      Valid IAGOS data saved to {iagos_valid_csv_path}")

    # Check if there are enough data points
    if len(ppb_values) < 2:
        print("        Error: Insufficient IAGOS data points, cannot process.")
        return None, None, None

    # Sort data by descending pressure (from high pressure to low pressure)
    sorted_indices = np.argsort(-pressure_values)
    ppb_values = ppb_values[sorted_indices]
    pressure_values = pressure_values[sorted_indices]

    # Calculate average PPB and DU within each OMI pressure layer
    avg_ppb_iagos = []
    du_iagos = []
    avg_pressure_iagos = []

    # Get the minimum pressure value from IAGOS data
    min_pressure_iagos = pressure_values.min()

    for i in range(len(pressure_boundaries_omi) - 1):
        P_top = pressure_boundaries_omi[i]
        P_bottom = pressure_boundaries_omi[i + 1]

        # If the bottom pressure of the current layer is less than the minimum pressure of IAGOS data, skip this layer
        if P_bottom < min_pressure_iagos:
            print(f"        Layer {i}: Bottom pressure {P_bottom:.2f} hPa is less than IAGOS minimum pressure {min_pressure_iagos:.2f} hPa, skipping this layer.")
            avg_ppb_iagos.append(np.nan)
            du_iagos.append(np.nan)
            avg_pressure_iagos.append((P_top + P_bottom) / 2)
            continue

        # Find indices of IAGOS data points within the current OMI pressure layer
        mask = (pressure_values <= P_top) & (pressure_values > P_bottom)
        if np.any(mask):
            ppb_in_layer = ppb_values[mask]
            pressure_in_layer = pressure_values[mask]

            # Calculate ΔP_i
            delta_p_in_layer = np.diff(pressure_in_layer, prepend=pressure_in_layer[0])

            # Calculate weighted average PPB
            weighted_ppb = np.sum(ppb_in_layer * delta_p_in_layer) / np.sum(delta_p_in_layer)
            avg_ppb_iagos.append(weighted_ppb)

            # Calculate pressure difference ΔP (hPa)
            delta_p = P_top - P_bottom

            # Calculate z_mid (km)
            z_mid = omi_height_layers[i]

            # Calculate DU
            du = (weighted_ppb * delta_p) / (1.251 * 1013.25) * ((earth_radius_km + z_mid) / earth_radius_km)**2
            du_iagos.append(du)

            # Calculate average pressure (for plotting)
            avg_pressure = (P_top + P_bottom) / 2
            avg_pressure_iagos.append(avg_pressure)
        else:
            avg_ppb_iagos.append(np.nan)
            du_iagos.append(np.nan)
            avg_pressure_iagos.append((P_top + P_bottom) / 2)

    avg_ppb_iagos = np.array(avg_ppb_iagos)
    du_iagos = np.array(du_iagos)
    avg_pressure_iagos = np.array(avg_pressure_iagos)

    print("        Resampled IAGOS average PPB values (before DU calculation):")
    print(avg_ppb_iagos)
    print("        Resampled IAGOS DU values:")
    print(du_iagos)
    print("        Resampled IAGOS average pressure values:")
    print(avg_pressure_iagos)

    # Check array lengths
    print("Lengths of arrays before creating DataFrame:")
    print(f"Length of avg_ppb_iagos: {len(avg_ppb_iagos)}")
    print(f"Length of du_iagos: {len(du_iagos)}")
    print(f"Length of avg_pressure_iagos: {len(avg_pressure_iagos)}")

    # Save resampled IAGOS data to CSV
    df_iagos_resampled = pd.DataFrame({
        'Layer_Index': np.arange(len(avg_ppb_iagos)),
        'Pressure_Boundary_Top_hPa': pressure_boundaries_omi[:-1],
        'Pressure_Boundary_Bottom_hPa': pressure_boundaries_omi[1:],
        'Average_Pressure_hPa': avg_pressure_iagos,
        'Average_Ozone_Concentration_ppb': avg_ppb_iagos,
        'DU': du_iagos,
    })
    iagos_resampled_csv_filename = f'IAGOS_Resampled_Data_{iagos_datetime_str}.csv'
    iagos_resampled_csv_path = os.path.join(csv_output_dir, iagos_resampled_csv_filename)
    df_iagos_resampled.to_csv(iagos_resampled_csv_path, index=False)
    print(f"      Resampled IAGOS data saved to {iagos_resampled_csv_path}")

    return avg_ppb_iagos, du_iagos, avg_pressure_iagos


In [ ]:
def plot_profiles(omi_data, iagos_data, output_path, omi_date_str, iagos_filename):
    """
    Plot the OMI and IAGOS ozone profiles over ICN Airport and save to the specified path.
    """
    (icn_ozprof_omi, icn_aozprof_omi, average_ps_hpa_omi, average_zs_km_omi,
     icn_rms_omi, icn_cfrac_omi, icn_avgres_omi, pressure_boundaries_omi) = omi_data
    (avg_ppb_iagos, du_iagos, avg_pressure_iagos) = iagos_data

    # Take the logarithm of average pressure for plotting
    log_average_ps_omi = np.log10(average_ps_hpa_omi)

    # Keep only the lowest 4 data points
    du_iagos_plot = du_iagos[:4]
    avg_pressure_iagos_plot = avg_pressure_iagos[:4]
    log_avg_pressure_iagos_plot = np.log10(avg_pressure_iagos_plot)

    # Print the resampled IAGOS data
    print("        Resampled IAGOS data (lowest 4 data points):")
    for idx in range(len(du_iagos_plot)):
        print(f"          DU[{idx+1}]: {du_iagos_plot[idx]:.4f}, Pressure[{idx+1}]: {avg_pressure_iagos_plot[idx]:.2f} hPa")

    # Create the figure
    fig, ax1 = plt.subplots(figsize=(10, 8))  # Increase figure size for better readability

    # Plot OMI ozone profile
    ax1.plot(icn_ozprof_omi, log_average_ps_omi, marker='o', linestyle='-', color='b', label='OMI O3 Concentration (ozprof)')

    # Plot OMI a priori ozone profile
    ax1.plot(icn_aozprof_omi, log_average_ps_omi, marker='x', linestyle='--', color='r', label='OMI A Priori O3 Concentration (aozprof)')

    # Plot the lowest 4 DU data points from IAGOS
    if np.any(~np.isnan(du_iagos_plot)):
        ax1.plot(du_iagos_plot, log_avg_pressure_iagos_plot, marker='s', linestyle='--', color='g', label='IAGOS O3 Concentration (DU)')
    else:
        print("        Warning: All of the lowest 4 IAGOS data points are NaN, not plotting this curve.")

    # Define RMS and Cloud Fraction information
    rms_text_win1 = f"RMS (win1): {icn_rms_omi[0]:.2f}"
    rms_text_win2 = f"RMS (win2): {icn_rms_omi[1]:.2f}"
    cfrac_text = f"Cloud Fraction: {icn_cfrac_omi:.2f}"

    # Define Avg Residual information
    avgres_text_win1 = f"Avg Residual (win1): {icn_avgres_omi[0]:.2f}"
    avgres_text_win2 = f"Avg Residual (win2): {icn_avgres_omi[1]:.2f}"

    # Create custom legend elements, including additional text information
    legend_elements = [
        Line2D([0], [0], color='b', marker='o', linestyle='-', label='OMI O3 Concentration (ozprof)'),
        Line2D([0], [0], color='r', marker='x', linestyle='--', label='OMI A Priori O3 Concentration (aozprof)'),
    ]

    if np.any(~np.isnan(du_iagos_plot)):
        legend_elements.append(Line2D([0], [0], color='g', marker='s', linestyle='--', label='IAGOS O3 Concentration (DU)'))

    # Add RMS, Cloud Fraction, and Avg Residual information
    legend_elements.extend([
        Line2D([0], [0], color='none', label=rms_text_win1),
        Line2D([0], [0], color='none', label=rms_text_win2),
        Line2D([0], [0], color='none', label=cfrac_text),
        Line2D([0], [0], color='none', label=avgres_text_win1),
        Line2D([0], [0], color='none', label=avgres_text_win2),
    ])

    # Add the legend
    ax1.legend(handles=legend_elements, loc='upper right', fontsize=10)

    # Set title and axis labels
    ax1.set_title(f'O3 Vertical Profile over ICN Airport (OMI & IAGOS) - {omi_date_str}', fontsize=14)
    ax1.set_xlabel('O3 (DU)', fontsize=12)

    # Set pressure axis ticks
    pressure_ticks_hpa = [1000, 100, 10, 1, 0.1]
    log_pressure_ticks = np.log10(pressure_ticks_hpa)
    ax1.set_yticks(log_pressure_ticks)
    ax1.set_yticklabels([f"{p}" for p in pressure_ticks_hpa])
    ax1.set_ylabel('Pressure (hPa)', fontsize=12)

    # Create a secondary y-axis for altitude
    ax2 = ax1.twinx()

    # Set altitude axis limits based on height data
    min_height = average_zs_km_omi.min()
    max_height = average_zs_km_omi.max()
    ax2.set_ylim(min_height, max_height)

    # Set fixed altitude ticks
    fixed_height_ticks = [0, 10, 20, 30, 40, 50, 60, 70]
    ax2.set_yticks(fixed_height_ticks)
    ax2.set_yticklabels([f"{h}" for h in fixed_height_ticks])
    ax2.set_ylabel('Height (km)', fontsize=12)

    # Add grid lines
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

    # Invert pressure axis so that high pressure is at the bottom
    ax1.invert_yaxis()
    # Do not invert altitude axis

    # Adjust layout to prevent labels from being cut off
    plt.tight_layout()

    # Save the figure
    plt.savefig(output_path, dpi=300)
    plt.show()
    plt.close()
